In [1]:
import pandas as pd

In [2]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling1D

In [3]:
import numpy as np

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [5]:
from gensim.models import Word2Vec, KeyedVectors

In [6]:
from string import punctuation

In [7]:
from nltk.tokenize import word_tokenize

In [8]:
word_embedding = KeyedVectors.load_word2vec_format("C:\\git\\dados\\nlp\\skip_s100\\skip_s100.txt")

In [9]:
df = pd.read_csv("C:\\git\\dados\\nlp\\imdb-reviews-pt-br.csv", encoding="utf-8")
df.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [10]:
df.dropna( inplace=True )
df[df["text_pt"].isna()]

,id,text_en,text_pt,sentiment


In [11]:
# Create a translation table to remove all punctuation
translation_table = str.maketrans("", "", punctuation)
df["texto_limpo"] = df["text_pt"].str.translate(translation_table).str.lower()
df.head()

,id,text_en,text_pt,sentiment,texto_limpo
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg,mais uma vez o sr costner arrumou um filme por...
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg,este é um exemplo do motivo pelo qual a maiori...
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg,primeiro de tudo eu odeio esses raps imbecis q...
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg,nem mesmo os beatles puderam escrever músicas ...
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg,filmes de fotos de latão não é uma palavra apr...


In [12]:
def tokenizar( texto ):
    return word_tokenize(texto, language='portuguese')

df["texto_final"] = df["texto_limpo"].apply( tokenizar )
df.head()

,id,text_en,text_pt,sentiment,texto_limpo,texto_final
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg,mais uma vez o sr costner arrumou um filme por...,"[mais, uma, vez, o, sr, costner, arrumou, um, ..."
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg,este é um exemplo do motivo pelo qual a maiori...,"[este, é, um, exemplo, do, motivo, pelo, qual,..."
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg,primeiro de tudo eu odeio esses raps imbecis q...,"[primeiro, de, tudo, eu, odeio, esses, raps, i..."
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg,nem mesmo os beatles puderam escrever músicas ...,"[nem, mesmo, os, beatles, puderam, escrever, m..."
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg,filmes de fotos de latão não é uma palavra apr...,"[filmes, de, fotos, de, latão, não, é, uma, pa..."


In [13]:
dimensoes = 100
# max_palavras = 0

max_palavras = 50
# for linha in df["texto_final"]:
#     if len(linha) > max_palavras:
#         max_palavras = len(linha)
# print("Texto com o maior numero de palavras: ", max_palavras)

In [14]:
vetor_vazio = np.zeros((dimensoes))
vetor_vazio

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [15]:
def transformar_tokens_vetores( tokens ):
    lista_vetores = []
    indice_token = 0
    while indice_token < max_palavras:
        vetor = vetor_vazio
        if indice_token < len(tokens):
            token = tokens[indice_token]
            try:
                vetor = word_embedding[token]
            except KeyError as err:
                pass
    
        lista_vetores.append( vetor )
        indice_token += 1
    
    # for _ in range( max_palavras - len(lista_vetores) ):
    #     lista_vetores.append( vetor_vazio )
    return lista_vetores

In [16]:
# def transformar_tokens_vetores( tokens ):
#     lista_vetores = []
#     for token in tokens:
#         vetor = vetor_vazio
#         try:
#             vetor = word_embedding[token]
#         except KeyError as err:
#             pass
    
#         lista_vetores.append( vetor )
    
#     for _ in range( max_palavras - len(lista_vetores) ):
#         lista_vetores.append( vetor_vazio )
#     return lista_vetores

In [17]:
# transformar_tokens_vetores( df["texto_final"][0] )

[array([-0.18531 , -0.100611, -0.002013, -0.054001, -0.096575, -0.152258,
        -0.012691, -0.047499,  0.055455,  0.313066, -0.203088, -0.058859,
        -0.103879, -0.155771,  0.246319,  0.033125,  0.109872,  0.303559,
        -0.222814,  0.13338 ,  0.171023,  0.17002 ,  0.109307, -0.132152,
         0.033375,  0.110856,  0.33938 ,  0.438386, -0.368144, -0.106577,
         0.105061, -0.038856, -0.023362,  0.04443 ,  0.015557,  0.240807,
         0.065114, -0.110322, -0.047674,  0.041299, -0.060861, -0.092991,
         0.231673, -0.34233 , -0.021077,  0.175154,  0.04433 ,  0.086424,
         0.076284, -0.157184,  0.082431,  0.134378, -0.026654, -0.011076,
         0.148547, -0.002442, -0.192288,  0.176924, -0.337533,  0.097747,
        -0.009934,  0.005246, -0.097464, -0.115727,  0.019888, -0.11666 ,
        -0.162224,  0.265154,  0.208547, -0.330214, -0.09775 ,  0.316894,
         0.037787,  0.525398, -0.134633, -0.149629,  0.26933 ,  0.081822,
         0.091815,  0.278142,  0.14447

In [18]:
df["X"] = df["texto_final"].apply( transformar_tokens_vetores )

In [19]:
df["X"].shape

(49459,)

In [20]:
df["Y"] = df["sentiment"].replace( {"neg": 0, "pos": 1} )

C:\Users\anton\AppData\Local\Temp\ipykernel_25356\396284922.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Y"] = df["sentiment"].replace( {"neg": 0, "pos": 1} )


In [21]:
Y = df["Y"].to_numpy().astype("float32").reshape(-1, 1)
X = np.stack(df["X"].to_numpy()).astype("float32")

In [22]:
X

array([[[-0.18531 , -0.100611, -0.002013, ..., -0.288438,  0.035926,
         -0.067056],
        [ 0.01856 , -0.29358 , -0.407523, ...,  0.141665,  0.332921,
          0.05711 ],
        [-0.2109  ,  0.268442, -0.056659, ...,  0.051535, -0.144633,
          0.609435],
        ...,
        [-0.160473, -0.187661, -0.039564, ..., -0.184596,  0.040386,
          0.132213],
        [-0.09889 , -0.106548, -0.110395, ..., -0.146739,  0.145833,
          0.185859],
        [ 0.046746, -0.048127, -0.159766, ..., -0.111041,  0.005637,
          0.089662]],

       [[-0.081764, -0.014319,  0.027425, ..., -0.259547, -0.056221,
         -0.059513],
        [-0.194589, -0.2721  , -0.050452, ..., -0.011145,  0.075908,
          0.358806],
        [-0.174197, -0.338936, -0.143474, ..., -0.288831,  0.05916 ,
          0.118766],
        ...,
        [-0.055351,  0.007896,  0.025346, ..., -0.083906,  0.040999,
          0.238978],
        [-0.23587 ,  0.14589 ,  0.011962, ..., -0.11104 , -0.022172,
   

In [23]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 100, test_size=0.2)

In [24]:
modelo = Sequential()
modelo.add( Input( (max_palavras, dimensoes, ) ) )
modelo.add( GlobalAveragePooling1D() )
modelo.add( Dense( 128, activation="relu" ) )
modelo.add( Dense( 16, activation="relu" ) )
modelo.add( Dense( 1, activation="sigmoid" ) )
modelo.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ global_average_pooling1d             │ (None, 100)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          12,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 16)                  │           2,064 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,009 (58.63 KB)

 Trainable params: 15,009 (58.63 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
modelo.compile( optimizer = "adam", loss="binary_crossentropy", metrics=["accuracy"] )

In [26]:
historico = modelo.fit( X_train, Y_train, epochs = 10, batch_size = 128 )

Epoch 1/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.6130 - loss: 0.6595
Epoch 2/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6657 - loss: 0.6133
Epoch 3/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6744 - loss: 0.6028
Epoch 4/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6730 - loss: 0.6014
Epoch 5/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6808 - loss: 0.5924
Epoch 6/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6823 - loss: 0.5916
Epoch 7/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6843 - loss: 0.5899
Epoch 8/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6801 - loss: 0.5901
Epoch 9/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6895 - loss: 0.5835
Epoch 10/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6865 - loss: 0.5835


In [27]:
modelo.evaluate( X_test, Y_test )

310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6774 - loss: 0.5899


[0.5869805812835693, 0.6834815740585327]

In [28]:
vetorizador_predict = CountVectorizer( max_features = max_palavras, vocabulary = vocabulario, binary = True )

NameError: name 'FEATURES' is not defined

In [ ]:
texto = "I love music"
bow_predict = vetorizador_predict.fit_transform( [ texto ] )
bow_predict_descompressed = pd.DataFrame.sparse.from_spmatrix( bow_predict , columns = vocabulario ).astype(float)
"positivo" if modelo.predict( bow_predict_descompressed )[0][0] >= 0.5 else "negativo"